In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


In [35]:
credit_card_data = pd.read_csv('/creditcard.csv')

In [36]:
credit_card_data.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [37]:
credit_card_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [38]:
#cheking distribution of legit transaction & frudulent transactions
credit_card_data['Class'].value_counts()
#hence our data set is highly unbalanced


0    284315
1       492
Name: Class, dtype: int64

In [39]:
#0 for genuine transaction
#1 for fraudulent transaction
# Now we are seperating data into 2 sets
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [40]:
#getting statistical mesaures of the data
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [41]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [42]:
# now comapring the values for both the transaction
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Under Sampling


In [43]:
legit_sample = legit.sample(n=492)

In [44]:
# Concatenating two DataFrames
new_dataset = pd.concat([legit_sample, fraud],axis=0)
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
220188,142061.0,2.189167,0.113004,-2.399328,0.116368,1.073105,-0.704057,0.674918,-0.443073,0.126688,...,0.045159,0.336461,-0.100222,0.068588,0.522787,0.725682,-0.115362,-0.085250,0.76,0
195352,131012.0,0.141640,1.023384,-0.970488,-0.090905,0.187117,-1.194280,0.437265,0.298136,-0.280508,...,0.377640,0.997902,-0.068814,-0.019107,-0.342348,-0.172121,-0.080322,-0.057631,15.00,0
283710,171808.0,-1.496943,0.808773,-0.038707,-3.292707,-0.580473,-0.945241,-0.120701,0.953563,1.484939,...,-0.136861,-0.503160,0.160439,0.612381,-0.428487,-0.817383,0.268443,0.147840,25.15,0
121453,76195.0,-0.242863,-1.233273,1.953026,-0.807789,-1.139294,-0.696013,-0.813326,-0.227073,-2.103268,...,0.093499,0.602626,0.183318,0.366476,-0.635739,0.029602,0.017868,-0.038883,78.00,0
110084,71672.0,1.275748,0.112113,-0.314506,0.156521,0.546293,0.503068,-0.029376,0.101389,-0.195345,...,-0.006103,0.030767,-0.221593,-1.295539,0.614376,0.492987,-0.029529,-0.016860,7.49,0


In [45]:
new_dataset['Class'].value_counts()

0    492
1    492
Name: Class, dtype: int64

In [46]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,95368.833333,0.007533,-0.005309,0.042241,-0.060900,0.029074,-0.096783,0.023526,0.065477,0.033692,...,-0.029797,-0.022064,0.007956,0.039717,0.002232,-0.016548,0.017814,0.022254,0.030429,78.797033
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [47]:
#Splitting the data into Features and Targets
X = new_dataset.drop(columns='Class',axis=1)
Y = new_dataset['Class']

In [48]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
220188  142061.0  2.189167  0.113004 -2.399328  0.116368  1.073105 -0.704057   
195352  131012.0  0.141640  1.023384 -0.970488 -0.090905  0.187117 -1.194280   
283710  171808.0 -1.496943  0.808773 -0.038707 -3.292707 -0.580473 -0.945241   
121453   76195.0 -0.242863 -1.233273  1.953026 -0.807789 -1.139294 -0.696013   
110084   71672.0  1.275748  0.112113 -0.314506  0.156521  0.546293  0.503068   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [49]:
print(Y)

220188    0
195352    0
283710    0
121453    0
110084    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [50]:
# Split the data into training and testing Data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2,stratify=Y,random_state=2)

In [51]:
print(X.shape,X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


Now we are training the model with Logistic traning model


In [52]:
model = LogisticRegression()
# traing the LRM with training data


In [56]:
model.fit(X_train, Y_train)

LogisticRegression()

In [59]:
# Model Evaluation
#Accuracy Score
#Accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [60]:
print('Accuracy on training data: ',training_data_accuracy)

Accuracy on training data:  0.9491740787801779


Accuracy check on test data

In [62]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [63]:
print('Accuracy score on test Data: ', test_data_accuracy)

Accuracy score on test Data:  0.9289340101522843
